# Data Science Essentials: Data Cleaning
    Daniel Perkins
    MATH 403
    10/15/24
    

In [1]:
import pandas as pd
import statsmodels.api as sm

### Problem 1

The g\_t\_results.csv file is a set of parent-reported scores on their child's Gifted and Talented tests. 
The two tests, OLSAT and NNAT, are used by NYC to determine if children are qualified for gifted programs.
The OLSAT Verbal has 16 questions for Kindergardeners and 30 questions for first, second, and third graders.
The NNAT has 48 questions. Each test assigns 1 point to each question asked (so there are no non integer scores).
Using this dataset, answer the following questions.



1) What column has the highest number of null values and what percent of its values are null? Print the answer as a tuple with (column name, percentage). Make sure the second value is a percent.

2) List the columns that should be numeric that aren't. Print the answer as a tuple.

3) How many third graders have scores outside the valid range for the OLSAT Verbal Score? Print the answer

4) How many data values are missing (NaN)? Print the number.


In [20]:
# 1) What column has the highest number of null values and what percent of its values are null? 
# Print the answer as a tuple with (column name, percentage). 
# Make sure the second value is a percent.

results = pd.read_csv("g_t_results.csv")  # Load in data
null_percents = results.isna().mean()  # Count missing values (as a mean)
answer = (null_percents.idxmax(), max(null_percents))
print("The column with the most null values is:")
print(answer)

The column with the most null values is:
('School Assigned', 0.7521367521367521)


In [22]:
# 2) List the columns that should be numeric that aren't. Print the answer as a tuple.

print(results.dtypes)  # Print out the data types
print()
wrong_type = ("OLSAT Verbal Score", "OLSAT Verbal Percentile", "NNAT Non Verbal Raw Score")

print("The columns that should be numeric but aren't are")
print(wrong_type)
# It may also be beneficial to make Timestamp, Grade Level, and Birth Month numeric (since they are ordinal)

Timestamp                      object
Entering Grade Level           object
District                      float64
Birth Month                    object
OLSAT Verbal Score             object
OLSAT Verbal Percentile        object
NNAT Non Verbal Raw Score      object
NNAT Non Verbal Percentile      int64
Overall Score                   int64
School Preferences             object
School Assigned                object
Will you enroll there?         object
dtype: object

The columns that should be numeric but aren't are
('OLSAT Verbal Score', 'OLSAT Verbal Percentile', 'NNAT Non Verbal Raw Score')


In [44]:
# 3) How many third graders have scores outside the valid range for the 
# OLSAT Verbal Score? Print the answer

# results["Entering Grade Level"].value_counts()
third_graders = results[results["Entering Grade Level"] == "3"]  # Just get third graders
    
scores_out_range = 0
for index, row in third_graders.iterrows():  # Iterate through the rows
    score = int(row["OLSAT Verbal Score"])
    if score < 0 or score > 30:  # If not in the valid range, add 1
        scores_out_range += 1
        
print(f"{scores_out_range} third grader(s) had OLSAT Verbal Scores that were out of the valid range")

1 third grader(s) had OLSAT Verbal Scores that were out of the valid range


In [48]:
# 4) How many data values are missing (NaN)? Print the number.
null_count = results.isna().sum()  # Count missing values of each column
null_count = null_count.sum()  # Add numbers from each column together
print(f"{null_count} values are missing from the data in total")

192 values are missing from the data in total


### Problem 2

imdb.csv contains a small set of information about 99 movies. Clean the data set by doing the following in order: 

1) Remove duplicate rows by dropping the first **or** last. Print the shape of the dataframe after removing the rows.

2) Drop all rows that contain missing data. Print the shape of the dataframe after removing the rows.

3) Remove rows that have data outside valid data ranges and explain briefly how you determined your ranges for each column.

4) Identify and drop columns with three or fewer different values. Print a tuple with the names of the columns dropped.

5) Convert the titles to all lower case.

Print the first five rows of your dataframe.

In [125]:
movies = pd.read_csv("imdb.csv")  # Load in data
print(f"Before Data Cleaning: {movies.shape}")

movies = movies.drop_duplicates(keep="first")
print(f"After removing duplicates: {movies.shape}")

movies = movies.dropna()
print(f"After removing missing data: {movies.shape}")

# I determined which values to remove by calling min and max until it made sense for each category
movies = movies[movies['duration'] >= 30] # Movies less than 30 minutes or more aren't really movies
movies = movies[movies['duration'] <= 500]
movies = movies[movies['gross'] >= 100000] # Some movies made way less than the rest
movies = movies[movies['title_year'] >= 2000]  # Movies before this are not in the dataset
movies = movies[movies['imdb_score'] >= 0]  # Movies can't have negative scores
movies = movies[movies['movie_facebook_likes'] >= 1000]  # Movies should have at least a few lies
print(f"After removing data outside of range: {movies.shape}")

# Remove columns with less 3 or less unique values
columns_to_remove = tuple(movies.columns[movies.nunique() <= 3])
movies = movies.loc[:, movies.nunique() > 3]
print(f"Dropped the columns: {columns_to_remove}")

# Make the titles lower case
movies["movie_title"] = movies["movie_title"].str.lower()

# Print the first five rows
print()
print("First Five Rows")
print(movies[:5])

Before Data Cleaning: (99, 13)
After removing duplicates: (93, 13)
After removing missing data: (64, 13)
After removing data outside of range: (55, 13)
Dropped the columns: ('color', 'language')

First Five Rows
       director_name  duration        gross  \
0    Martin Scorsese       240  116866727.0   
1        Shane Black       195  408992272.0   
2  Quentin Tarantino       187   54116191.0   
4      Peter Jackson       186  258355354.0   
8        Joss Whedon       173  623279547.0   

                                 genres                          movie_title  \
0          Biography|Comedy|Crime|Drama              the wolf of wall street   
1               Action|Adventure|Sci-Fi                           iron man 3   
2  Crime|Drama|Mystery|Thriller|Western                    the hateful eight   
4                     Adventure|Fantasy  the hobbit: the desolation of smaug   
8               Action|Adventure|Sci-Fi                         the avengers   

   title_year country   

### Problem 3
Load housing.csv into a dataframe with index=0. Descriptions of the features are in housing_data_description.txt for your convenience.  
The goal is to construct a regression model that predicts SalePrice using the other features of the dataset.  Do this as follows:

	1) Identify and handle the missing data.  Hint: Dropping every row with some missing data is not a good choice because it gives you an empty dataframe.  What can you do instead?
    
    2) Add two new features: 
		a) Remodeled: Whether or not a house has been remodeled with a Y if it has been
		   remodeled, or a N if it has not.
		
		b) TotalPorch: Using the 5 different porch/deck columns, create a new column that
		   provides the total square footage of all the decks and porches for each house.
    
	3) Identify the variable with nonnumeric values that are misencoded as numbers.  One-hot encode it. Hint: don't forget to remove one of the encoded columns to prevent collinearity with the constant column (which you will add later).
    
    4) Add a constant column to the dataframe.

    5) Save a copy of the dataframe.

	6) Choose four categorical featrues that seem very important in predicting SalePrice. One-hot encode these features and remove all other categorical features.
		
	7) Run an OLS using all numerical data regression on your model.  

	
Print the ten features that have the highest coef in your model. Then print the summary. Don't print the OLS

### Problem 4

Using the copy of the dataframe you created in Problem 3, one-hot encode all the categorical variables.
Print the shape of the dataframe and run OLS.

Print the ten features that have the highest coef in your model and the summary.
Write a couple of sentences discussing which model is better and why.